# Building A chatbot with LangGraph
LangGraph offers Message Persistence through a built-in persistence layer.
- Ideal for chat applications that support multiple conversational turns

Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications

In [1]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import init_chat_model

In [2]:
TEMPERATURE = 0.0
NUM_PREDICT = 256
MODEL = "gemma3:12b-it-qat"

model = init_chat_model(
    model=MODEL,
    temperature=TEMPERATURE,
    num_predict=NUM_PREDICT,
    use_gpu=True,
    model_provider='ollama',
)

In [3]:
# Define a new Graph
workflow = StateGraph(state_schema=MessagesState)

In [4]:
def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {"messages": response}

In [5]:
# Define the single node in the graph
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add Memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a thread_id. This should look like:

In [6]:
config = {"configurable": {"thread_id": "abc123"}}

In [7]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. I'm a large language model. What can I do for you today?


In [8]:
query = "What is my name."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Your name is Bob! You told me that at the beginning of our conversation. 😊


In [11]:
for text in output['messages']:
    text.pretty_print()

================================ Human Message =================================

Hi! I'm Bob.
================================== Ai Message ==================================

Hi Bob! It's nice to meet you. I'm a large language model. What can I do for you today?
================================ Human Message =================================

What is my name.
================================== Ai Message ==================================

Your name is Bob! You told me that at the beginning of our conversation. 😊
